In [1]:

import bigframes.pandas as bpd
bpd.options.bigquery.project = "ml-product-study"

In [2]:
from bigframes.ml.llm import GeminiTextGenerator

model = GeminiTextGenerator()

/home/jupyter/.local/lib/python3.10/site-packages/bigframes/pandas/__init__.py:856: DefaultLocationWarning: No explicit location is set, so using location US for the session.
  return global_session.get_global_session()


In [3]:
# Define schema for the 'customers' table
customers_schema = """
Column name: customer_id, type: int, Description: Unique identifier for each customer
Column name: Name, type: string, Description: Customer's full name
Column name: Email, type: string, Description: Customer's email address
"""

# Define schema for the 'orders' table
orders_schema = """
Column name: order_id, type: int, Description: Unique identifier for each order
Column name: customer_id, type: int, Description: Foreign key referencing the 'customers' table
Column name: order_date, type: string, Description: Date of the order
Column name: order_amount, type: float, Description: Total amount of the order
"""

# Generate synthetic data for the 'customers' table
prompt_customers = f"""
Write python code to generate a pandas dataframe based on the requirements:
{customers_schema}

Note:
  - Return the code only, no additional texts or comments
  - Use faker library
  - Generate 100 rows
  - The final dataframe should be named 'customers_df'.
"""

df_prompt_customers = bpd.DataFrame({"prompt": [prompt_customers]})

In [4]:
max_tries = 5
for i in range(max_tries):
    # Get LLM generated code for 'customers' table
    df_result_customers = model.predict(df_prompt_customers)
    llm_result_customers = df_result_customers['ml_generate_text_llm_result'].iloc[0]

    # Python code comes back as a markdown code block,
    # remove the prefix "```python" and suffix "```"
    code_customers = llm_result_customers[9:-3]
    print(code_customers)

    # Check if the generated code is runnable
    try:
        exec(code_customers)
        break
    except Exception as ex:
        print(ex)
        error_context = f"""
Previous code:
{code_customers}

Had this exception:
{ex}"""

        # Update the prompt to help LLM correct error
        df_prompt_customers["prompt"] += error_context

        # If we have exhausted max tries then stop trying
        if i + 1 == max_tries:
            raise Exception("Failed to generate runnable code for 'customers' table")

execution_context_customers = {}
exec(code_customers, execution_context_customers)
customers_df = execution_context_customers.get("customers_df")

/home/jupyter/.local/lib/python3.10/site-packages/bigframes/core/__init__.py:92: PreviewWarning: Interpreting JSON column(s) as StringDtype. This behavior may change in future versions.
  warnings.warn(



import pandas as pd
from faker import Faker

# Create a faker instance
fake = Faker()

# Generate customer data
customer_id = [i for i in range(1, 101)]
names = [fake.name() for _ in range(100)]
emails = [fake.email() for _ in range(100)]

# Create the pandas dataframe
customers_df = pd.DataFrame({
    'customer_id': customer_id,
    'Name': names,
    'Email': emails
})

# Print the dataframe
print(customers_df)
`
invalid syntax (<string>, line 22)


/home/jupyter/.local/lib/python3.10/site-packages/bigframes/core/__init__.py:92: PreviewWarning: Interpreting JSON column(s) as StringDtype. This behavior may change in future versions.
  warnings.warn(



import pandas as pd
from faker import Faker

# Create a Faker instance
fake = Faker()

# Generate customer data
data = [
    {
        "customer_id": i,
        "Name": fake.name(),
        "Email": fake.email(),
    }
    for i in range(1, 101)
]

# Create the pandas dataframe
customers_df = pd.DataFrame(data)

# Set column types
customers_df = customers_df.astype(
    {"customer_id": "int", "Name": "string", "Email": "string"}
)

print(customers_df)

    customer_id                 Name                        Email
0             1         Raymond Ford           rjones@example.org
1             2         Bradley Bell           eric58@example.org
2             3    Stephanie Maxwell         khanjuan@example.com
3             4           Paul Allen  anthonymartinez@example.net
4             5  Christopher Stewart        richard04@example.com
..          ...                  ...                          ...
95           96      Nicole Bradford        yanderson@example.org
96           9

In [5]:
# Generate synthetic data for the 'orders' table
prompt_orders = f"""
Write python code to generate a pandas dataframe based on the requirements:
{orders_schema}

Note:
  - Return the code only, no additional texts or comments
  - Use faker library
  - Generate 100 rows
  - The final dataframe should be named 'orders_df'.
  - The 'customer_id' column should reference existing customer IDs from the 'customers_df' dataframe.
"""

df_prompt_orders = bpd.DataFrame({"prompt": [prompt_orders]})

max_tries = 5
for i in range(max_tries):
    # Get LLM generated code for 'orders' table
    df_result_orders = model.predict(df_prompt_orders)
    llm_result_orders = df_result_orders['ml_generate_text_llm_result'].iloc[0]

    # Python code comes back as a markdown code block,
    # remove the prefix "```python" and suffix "```"
    code_orders = llm_result_orders[9:-3]
    print(code_orders)

    # Check if the generated code is runnable
    try:
        # Make 'customers_df' available in the execution context for 'orders' table generation
        execution_context_orders = {"customers_df": customers_df}
        exec(code_orders, execution_context_orders)
        break
    except Exception as ex:
        print(ex)
        error_context = f"""
Previous code:
{code_orders}

Had this exception:
{ex}"""

        # Update the prompt to help LLM correct error
        df_prompt_orders["prompt"] += error_context

        # If we have exhausted max tries then stop trying
        if i + 1 == max_tries:
            raise Exception("Failed to generate runnable code for 'orders' table")

orders_df = execution_context_orders.get("orders_df")


/home/jupyter/.local/lib/python3.10/site-packages/bigframes/core/__init__.py:92: PreviewWarning: Interpreting JSON column(s) as StringDtype. This behavior may change in future versions.
  warnings.warn(



import pandas as pd
from faker import Faker

fake = Faker()

# Generate fake data for the dataframe
data = [
    {
        "order_id": fake.unique.random_int(min=1, max=10000),
        "customer_id": fake.random.choice(customers_df["customer_id"].tolist()),
        "order_date": fake.date(),
        "order_amount": fake.pydecimal.Decimal(min_value=100, max_value=10000),
    }
    for _ in range(100)
]

# Create a pandas dataframe
orders_df = pd.DataFrame(data)

# Set the data types for the columns
orders_df["order_id"] = orders_df["order_id"].astype("int")
orders_df["customer_id"] = orders_df["customer_id"].astype("int")
orders_df["order_date"] = pd.to_datetime(orders_df["order_date"])
orders_df["order_amount"] = pd.to_numeric(orders_df["order_amount"])

'function' object has no attribute 'Decimal'


/home/jupyter/.local/lib/python3.10/site-packages/bigframes/core/__init__.py:92: PreviewWarning: Interpreting JSON column(s) as StringDtype. This behavior may change in future versions.
  warnings.warn(



import pandas as pd
from faker import Faker

fake = Faker()

# Generate fake data for the dataframe
data = [
    {
        "order_id": fake.unique.random_int(min=1, max=10000),
        "customer_id": fake.random.choice(customers_df["customer_id"].tolist()),
        "order_date": fake.date(),
        "order_amount": fake.pyfloat(min_value=100, max_value=10000),
    }
    for _ in range(100)
]

# Create a pandas dataframe
orders_df = pd.DataFrame(data)

# Set the data types for the columns
orders_df["order_id"] = orders_df["order_id"].astype("int")
orders_df["customer_id"] = orders_df["customer_id"].astype("int")
orders_df["order_date"] = pd.to_datetime(orders_df["order_date"])
orders_df["order_amount"] = pd.to_numeric(orders_df["order_amount"])



In [6]:
# Print the generated dataframes
print("Customers DataFrame:")
print(customers_df)

print("\nOrders DataFrame:")
print(orders_df)

Customers DataFrame:
    customer_id               Name                       Email
0             1      Allison Baird    rebekahfloyd@example.org
1             2     Cesar Mitchell          esmith@example.com
2             3  Courtney Mcdonald     aarondavies@example.net
3             4        Sherri Cobb          hlopez@example.net
4             5  Gregory Zimmerman  caitlinelliott@example.com
..          ...                ...                         ...
95           96      Stanley Smith         pwarner@example.org
96           97      Sarah Mcclain   torresmichael@example.com
97           98     Charles Rivera       william19@example.com
98           99   Melissa Gonzalez     aliciagates@example.com
99          100         Jenna Long      victoria58@example.net

[100 rows x 3 columns]

Orders DataFrame:
    order_id  customer_id order_date  order_amount
0        628           98 2013-08-31   4268.351124
1       1957           37 2003-04-17   4692.799000
2       1736           16 2